In [1]:
import numpy as np
import xarray as xr
import os
import sys
sys.path.append("../")
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

from src.data_utils import *        

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
data_dir = "../data/"

time_start = "2017-01-01"
time_end = "2022-12-31"

zarr_ds = load_data(data_dir, time_start=time_start, time_end=time_end)

In [3]:
batch_size = 32
train_loader, test_loader = get_data_loaders(zarr_ds, 0.8, batch_size=batch_size)

In [4]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [5]:
x, y = next(iter(train_loader))
x = x.to(device)

In [49]:
from src.vae_model import *
from tqdm import tqdm

In [52]:
model = ConvVAE(input_channels=10, latent_dim=8).to(device)
model.compile()
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
kl_lambda = 0.5

In [53]:
num_epochs = 100

train_rec_losses, train_kl_losses, val_losses = train(
    model,
    opt,
    train_loader,
    test_loader,
    device,
    kl_lambda=kl_lambda,
    num_epochs=num_epochs,
    show_every=10,
)

Training epoch 1:   0%|          | 0/55 [00:00<?, ?it/s]

Training epoch 1:  98%|█████████▊| 54/55 [01:33<00:01,  1.20s/it, Loss=0.1226, rec. loss=0.1036, KL loss=0.0381]/gscratch/rao/aresf/miniconda3/envs/torch/lib/python3.11/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/gscratch/rao/aresf/miniconda3/envs/torch/lib/python3.11/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/gscratch/rao/aresf/miniconda3/envs/torch/lib/python3.11/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/gscratch/rao/aresf/miniconda3/envs/torch/lib/python3.11/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


KeyboardInterrupt: 